# Bibliotecas 

In [1]:
import pandas as pd
import numpy as np
import re
from pycaret.classification import *

# Importação das Informações 

In [2]:
#importação das partidas 
partidas_2017 = pd.read_csv('data/2017_partidas.csv')
partidas_2016 = pd.read_csv('data/2016_partidas.csv')
partidas_2015 = pd.read_csv('data/2015_partidas.csv')
partidas_2014 = pd.read_csv('data/2014_partidas.csv')

In [3]:
#importação dos scouts
scouts_2017 = pd.read_csv('data/2017_scouts.csv')
scouts_2016 = pd.read_csv('data/2016_scouts.csv')
scouts_2015 = pd.read_csv('data/2015_scouts.csv')
scouts_2014 = pd.read_csv('data/2014_scouts.csv')

In [4]:
#importação dos clubes 
clubes_2017 = pd.read_csv('data/2017_clubes.csv')
clubes_2016 = pd.read_csv('data/2016_clubes.csv')
clubes_2015 = pd.read_csv('data/2015_clubes.csv')
clubes_2014 = pd.read_csv('data/2014_clubes.csv')

In [5]:
#importação dos atletas
atletas_2017 = pd.read_csv('data/2017_atletas.csv')
atletas_2016 = pd.read_csv('data/2016_atletas.csv')
atletas_2015 = pd.read_csv('data/2015_atletas.csv')
atletas_2014 = pd.read_csv('data/2014_atletas.csv')

# Junção das Tabelas

In [6]:
time_stats = scouts_2017.groupby(['rodada_id','clube_id']).agg('sum')

In [7]:
time_stats.reset_index(inplace=True)

In [8]:
time_stats

,rodada_id,clube_id,atleta_id,posicao_id,status_id,pontos_num,preco_num,variacao_num,media_num,jogos_num,FS,PE,A,FT,FD,FF,G,I,PP,RB,FC,GC,CA,CV,SG,DD,DP,GS
0,0,262,3150098,143,237,0.00,259.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,263,3116434,133,245,0.00,217.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,264,2849621,138,235,0.00,242.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,265,3084300,130,214,0.00,160.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,266,2917880,137,233,0.00,218.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,267,2375253,125,222,0.00,197.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,275,2505298,118,211,0.00,291.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,276,2721982,127,220,0.00,247.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,277,2958559,131,218,0.00,250.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,282,3135696,144,253,0.00,263.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
time_stats_filtrada = time_stats[['rodada_id','clube_id','preco_num']]

In [10]:
clubes_casa = time_stats_filtrada.rename(columns={'clube_id': 'clube_casa_id', 'preco_num':'preco_num_casa'})

In [11]:
clubes_visitante = time_stats_filtrada.rename(columns={'clube_id': 'clube_visitante_id','preco_num':'preco_num_visitante'})

In [12]:
clubes_casa['rodada_id'] = clubes_casa['rodada_id'] + 1
clubes_visitante['rodada_id'] = clubes_visitante['rodada_id'] + 1

In [13]:
tabela_casa = pd.merge(left = partidas_2017, right= clubes_casa, how = 'left', on = ['rodada_id','clube_casa_id'])
tabela_completa = pd.merge(left = tabela_casa, right= clubes_visitante, how = 'left', on = ['rodada_id','clube_visitante_id'])

In [14]:
partidas = tabela_completa

# Transformação de Dados

In [15]:
#como no dataset de partidas não fala o resultado(vitória, empate ou derrota)
partidas['resultado'] = 0
partidas.loc[partidas['placar_oficial_mandante'] > partidas['placar_oficial_visitante'],'resultado'] = 1
partidas.loc[partidas['placar_oficial_mandante'] < partidas['placar_oficial_visitante'],'resultado'] = -1

In [16]:
#transformação dos aproveitamentos mandante
partidas['vitorias_recentes_mandante'] = partidas['aproveitamento_mandante'].str.count('v')
partidas['empates_recentes_mandante'] = partidas['aproveitamento_mandante'].str.count('e')
partidas['derrotas_recentes_mandante'] = partidas['aproveitamento_mandante'].str.count('d')

#transformação dos aproveitamentos visitante
partidas['vitorias_recentes_visitante'] = partidas['aproveitamento_visitante'].str.count('v')
partidas['empates_recentes_visitante'] = partidas['aproveitamento_visitante'].str.count('e')
partidas['derrotas_recentes_visitante'] = partidas['aproveitamento_visitante'].str.count('d')

In [17]:
#transformação para int64 da coluna valida
partidas['valida'] = partidas['valida'].astype('int64')

In [18]:
#removendo vazamento de dados e dados já transformados
partidas.drop(columns=['aproveitamento_visitante','aproveitamento_mandante','placar_oficial_visitante','placar_oficial_mandante','valida'], inplace=True)

In [19]:
#remoção de colunas inuteis para o modelo
partidas.drop(columns=['partida_data', 'local','clube_casa_id','clube_visitante_id'],inplace=True)

In [20]:
partidas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   rodada_id                    200 non-null    int64  
 1   clube_casa_posicao           200 non-null    int64  
 2   clube_visitante_posicao      200 non-null    int64  
 3   preco_num_casa               200 non-null    float64
 4   preco_num_visitante          200 non-null    float64
 5   resultado                    200 non-null    int64  
 6   vitorias_recentes_mandante   200 non-null    int64  
 7   empates_recentes_mandante    200 non-null    int64  
 8   derrotas_recentes_mandante   200 non-null    int64  
 9   vitorias_recentes_visitante  200 non-null    int64  
 10  empates_recentes_visitante   200 non-null    int64  
 11  derrotas_recentes_visitante  200 non-null    int64  
dtypes: float64(2), int64(10)
memory usage: 20.3 KB


In [21]:
#transformação para int32 do dataframe
partidas = partidas.astype('int32')

# Modelagem

In [23]:
classification = setup(data = partidas, target='resultado',session_id=42)

,Description,Value
0,session_id,42
1,Target,resultado
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(200, 12)"
5,Missing Values,False
6,Numeric Features,11
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [24]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.6473,0.8183,0.6337,0.6902,0.6481,0.4635,0.4800,0.0030
lr,Logistic Regression,0.6401,0.8181,0.6243,0.6723,0.6389,0.4504,0.4635,0.3580
nb,Naive Bayes,0.6324,0.8202,0.6143,0.6750,0.6265,0.4351,0.4497,0.0040
ridge,Ridge Classifier,0.6319,0.0000,0.6076,0.6255,0.6111,0.4302,0.4500,0.0050
gbc,Gradient Boosting Classifier,0.6258,0.7973,0.5908,0.6509,0.6094,0.4211,0.4388,0.0510
lightgbm,Light Gradient Boosting Machine,0.5967,0.8082,0.5652,0.5975,0.5812,0.3753,0.3884,0.0120
et,Extra Trees Classifier,0.5901,0.7697,0.5579,0.5823,0.5675,0.3604,0.3730,0.0450
rf,Random Forest Classifier,0.5830,0.7922,0.5548,0.5902,0.5639,0.3560,0.3711,0.0470
ada,Ada Boost Classifier,0.5676,0.7883,0.5404,0.5720,0.5585,0.3341,0.3414,0.0220
dt,Decision Tree Classifier,0.5670,0.6759,0.5449,0.5821,0.5540,0.3346,0.3499,0.0030


In [29]:
print(best_model)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)


In [32]:
model = create_model('lda')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7143,0.7982,0.6944,0.7109,0.7072,0.5556,0.5602
1,0.5000,0.7804,0.5000,0.5500,0.5111,0.2576,0.2636
2,0.7857,0.9268,0.7778,0.8776,0.7879,0.6769,0.7160
3,0.5714,0.8089,0.5556,0.6000,0.5815,0.3538,0.3566
4,0.7857,0.8607,0.7500,0.7721,0.7732,0.6667,0.6722
5,0.6429,0.8286,0.6389,0.6762,0.6464,0.4615,0.4725
6,0.7143,0.8299,0.6444,0.6888,0.6956,0.5447,0.5495
7,0.4286,0.7174,0.4111,0.4536,0.4378,0.1318,0.1328
8,0.7143,0.8757,0.8095,0.8776,0.7143,0.5942,0.6721


In [27]:
evaluate_model(model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [28]:
final_model = finalize_model(model)